In [19]:
import pandas as pd
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_stock import DB_Stock
from db_nyse import DB_NYSE
from commonHelper import EDateType, EFinancialStatementType
from datetime import datetime
import yfinance as yf
from portfolio import Portfolio
import numpy as np

pd.set_option('display.max_rows', None)  # 모든 행 출
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.expand_frame_repr', False)  # 긴 데이터 프레임 줄바꿈 없이 출력
pd.set_option('display.max_colwidth', None)  # 열 하나에 들어가는 텍스트의 최대 길이를 제한 없이 설정
pd.set_option('display.width', None)         # 줄바꿈 없이 전체 데이터 프레임을 한 줄로 출력


# 범용 스코어 함수
def score_positive(val, min_val, max_val):
    if pd.isnull(val): return 0
    return max(0, min(10, 10 * (val - min_val) / (max_val - min_val)))

# 산업군별 지표별 min/max 설정
sector_metric_ranges = {
    "PER": {
        "Technology": (10, 100),
        "Financial Services": (5, 20),
        "Consumer Cyclical": (8, 40),
        "Healthcare": (10, 60),
        "Communication Services": (10, 50),
        "Industrials": (8, 30),
        "Consumer Defensive": (10, 25),
        "Energy": (5, 20),
        "Basic Materials": (5, 25),
        "Real Estate": (5, 30),
        "Utilities": (5, 25)
    },
    "PSR": {
        "Technology": (0.5, 20),
        "Financial Services": (0.5, 5),
        "Consumer Cyclical": (0.5, 6),
        "Healthcare": (0.5, 8),
        "Communication Services": (0.5, 6),
        "Industrials": (0.5, 5),
        "Consumer Defensive": (0.5, 5),
        "Energy": (0.5, 3),
        "Basic Materials": (0.5, 4),
        "Real Estate": (0.5, 8),
        "Utilities": (0.5, 4)
    },
    "DebtToEquityRatio": {
        "Technology": (0, 1),
        "Financial Services": (0, 10),
        "Consumer Cyclical": (0, 2),
        "Healthcare": (0, 1.5),
        "Communication Services": (0, 1.5),
        "Industrials": (0, 2),
        "Consumer Defensive": (0, 1.5),
        "Energy": (0, 2),
        "Basic Materials": (0, 2),
        "Real Estate": (0, 3),
        "Utilities": (0, 2)
    },
    "EV/EBIT": {
        "Technology": (5, 80),
        "Financial Services": (5, 20),
        "Consumer Cyclical": (5, 30),
        "Healthcare": (5, 50),
        "Communication Services": (5, 40),
        "Industrials": (5, 25),
        "Consumer Defensive": (5, 20),
        "Energy": (3, 15),
        "Basic Materials": (3, 20),
        "Real Estate": (5, 25),
        "Utilities": (4, 15)
    },
    "PBR": {
        "Technology": (2, 15),
        "Financial Services": (0.5, 2),
        "Consumer Cyclical": (1, 5),
        "Healthcare": (2, 6),
        "Communication Services": (1, 6),
        "Industrials": (1, 4),
        "Consumer Defensive": (1, 4),
        "Energy": (0.5, 2),
        "Basic Materials": (0.5, 2),
        "Real Estate": (0.5, 3),
        "Utilities": (0.5, 2)
    },
    "GP/A": {
        "Technology": (0.2, 0.7),
        "Financial Services": (0.05, 0.2),
        "Consumer Cyclical": (0.1, 0.4),
        "Healthcare": (0.1, 0.5),
        "Communication Services": (0.1, 0.4),
        "Industrials": (0.05, 0.25),
        "Consumer Defensive": (0.05, 0.25),
        "Energy": (0.05, 0.2),
        "Basic Materials": (0.05, 0.2),
        "Real Estate": (0.01, 0.1),
        "Utilities": (0.01, 0.1)
    }
}

# 기준값 가져오는 함수
def get_sector_range(sector, metric, default_min=0, default_max=10):
    return sector_metric_ranges.get(metric, {}).get(sector, (default_min, default_max))

# 스코어링 본체
def score_financials_by_sector_extended(df):
    df["CommonStockDividendPaid"] = df["CommonStockDividendPaid"].fillna(0)
    df["MarketCap"] = df["MarketCap"].fillna(0)
    df["DividendYieldProxy"] = df["CommonStockDividendPaid"] / df["MarketCap"]

    def row_score(row):
        sector = row["Sector"]
        r = {}

        r["IncomeGrowth_score"] = score_positive(row["IncomeGrowth"], -1, 5)

        psr_min, psr_max = get_sector_range(sector, "PSR", 0.5, 20)
        r["PSR_score"] = score_positive(psr_max - row["PSR"], 0, psr_max - psr_min)

        gp_min, gp_max = get_sector_range(sector, "GP/A", 0, 0.7)
        r["GP/A_score"] = score_positive(row["GP/A"], gp_min, gp_max)

        ev_min, ev_max = get_sector_range(sector, "EV/EBIT", 5, 400)
        r["EV/EBIT_score"] = score_positive(ev_max - row["EV/EBIT"], 0, ev_max - ev_min)

        per_min, per_max = get_sector_range(sector, "PER", 10, 50)
        r["PER_score"] = score_positive(per_max - row["PER"], 0, per_max - per_min)

        r["CurrentRatio_score"] = score_positive(row["CurrentRatio"], 1, 3)

        pbr_min, pbr_max = get_sector_range(sector, "PBR", 0.5, 25)
        r["PBR_score"] = score_positive(pbr_max - row["PBR"], 0, pbr_max - pbr_min)

        de_min, de_max = get_sector_range(sector, "DebtToEquityRatio", 0, 1)
        r["DebtToEquity_score"] = score_positive(de_max - row["DebtToEquityRatio"], 0, de_max - de_min)

        r["PCR_score"] = score_positive(150 - row["PCR"], 0, 140)
        r["PFCR_score"] = score_positive(150 - row["PFCR"], 0, 140)
        r["Dividend_score"] = score_positive(row["DividendYieldProxy"], 0, 0.05)

        r["TotalScore"] = sum(r.values()) / len(r)
        return pd.Series(r)

    scored_df = df.copy()
    score_result = df.apply(row_score, axis=1)
    return pd.concat([scored_df[["Date"]], score_result], axis=1)


with DB_FinancialStatement() as fs:
    df = fs.get_fs_all(['MRNA'], EDateType.YEAR)
    df = fs.get_sector(df)
    df = fs.get_marketCap(df)
    df = fs.get_psr(df)
    df = fs.get_gp_a(df)
    df = fs.get_por(df)
    df = fs.get_ev_ebti(df)
    df = fs.get_per(df)
    df = fs.get_liquidation_value(df)
    df = fs.get_current_ratio(df)
    df = fs.get_pbr(df)
    df = fs.get_debt_to_equity_ratio(df)
    df = fs.get_pcr(df)
    df = fs.get_pfcr(df)
    df = fs.get_dividend_payout_ratio(df)
    df = fs.get_income_growth(df)
    df = df[['Date','Symbol', 'Sector', 'MarketCap', 'OperatingIncome', 'IncomeGrowth', 'PSR', 'GP/A', 'POR', 'EV/EBIT', 'PER', 'LiquidationValue', 'CurrentRatio','PBR', 'DebtToEquityRatio', 'CommonStockDividendPaid', 'PCR', 'PFCR', 'DividendPayoutRatio']]
    df = df.dropna(subset=["MarketCap"])
    # df = score_financials_by_sector_extended(df)
    display(df)
    



,Date,Symbol,Sector,MarketCap,OperatingIncome,IncomeGrowth,PSR,GP/A,POR,EV/EBIT,PER,LiquidationValue,CurrentRatio,PBR,DebtToEquityRatio,CommonStockDividendPaid,PCR,PFCR,DividendPayoutRatio
1,2021-12-31,MRNA,Healthcare,102353940000.0,1.329600e+10,NaN,5.770971,0.612874,7.6981,6.960209,8.388292,5547000000.0,1.760627,7.236051,0.064758,None,7.514974,7.361474,None
2,2022-12-31,MRNA,Healthcare,69153700000.0,9.420000e+09,-0.29,3.663772,0.520497,7.341157,6.417378,8.269995,6696000000.0,2.728215,3.616258,0.062752,None,13.883497,12.851459,None
3,2023-12-31,MRNA,Healthcare,37989900000.0,-4.239000e+09,-1.45,5.6248,0.111853,-8.961996,-7.845517,-8.058952,5753000000.0,3.424544,2.742161,0.089721,None,None,None,None
4,2024-12-31,MRNA,Healthcare,16049880000.0,-3.945000e+09,-0.07,5.017155,0.122684,-4.068411,-2.72729,-4.507127,4858000000.0,3.671351,1.472331,0.068526,None,None,None,None
